In [5]:
!source venv/bin/activate
!pip3 install matplotlib seaborn pandas

  Using cached matplotlib-3.10.6-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached contourpy-1.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (112 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached

In [9]:
#importing libraries

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

In [23]:
BASE_DIR = Path("/data/tennis_atp")
# Patterns to include; extend if you also have JSON/Parquet
INCLUDE_PATTERNS = [
    "atp_matches*.csv",
    "atp_future_matches*.csv",
    "atp_matches*.parquet",
    "atp_future_matches*.parquet",
    "atp_matches*.json",
    "atp_future_matches*.json",
]
EXCLUDE_SUBSTRINGS = ["amateur","doubles"]  # skip any file containing these substrings in the name

In [29]:
print(files.extend(BASE_DIR.glob(pat)))

None


In [39]:
from pathlib import Path
import pandas as pd

# ---- Config ----
BASE_DIR = Path.home() / "api/lilly/data/tennis_atp"
INCLUDE_PATTERNS = ["atp_matches_*.csv", "atp_matches_futures_*.csv"]
EXCLUDE_SUBSTRINGS = ["amateur", "doubles", "qual_chall"]

def load_dataframe_from_file(fp: Path) -> pd.DataFrame:
    df = pd.read_csv(fp)
    # Always return a DataFrame; even empty is OK
    if not isinstance(df, pd.DataFrame):
        raise ValueError("read_csv did not return a DataFrame")
    df.attrs["source_file"] = fp.name
    return df

# Gather files
files = []
for pat in INCLUDE_PATTERNS:
    files.extend(BASE_DIR.glob(pat))
files = sorted(f for f in files if all(ex not in f.name.lower() for ex in EXCLUDE_SUBSTRINGS))

print(f"Found {len(files)} files to load.")

dfs, bad_files = [], []
for f in files:
    try:
        df = load_dataframe_from_file(f)
        dfs.append(df)
    except Exception as e:
        bad_files.append((f.name, str(e)))

print(f"Loaded {len(dfs)} dataframes.")
if bad_files:
    print("Bad files:")
    for name, err in bad_files[:10]:
        print(" •", name, "->", err)
    if len(bad_files) > 10:
        print(" ...")

# Summary (safe for empty frames)
summary_rows = []
for df in dfs:
    fname = df.attrs.get("source_file", "UNKNOWN")
    row = {"file": fname, "rows": len(df), "cols": len(df.columns)}
    # Try to parse a date-ish column
    date_col = next((c for c in ["tourney_date","date","match_date","start_date","utc_date"] if c in df.columns), None)
    if date_col:
        dt = pd.to_datetime(df[date_col], errors="coerce", utc=True)
        row["date_min"] = dt.min()
        row["date_max"] = dt.max()
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows).sort_values("file", ignore_index=True)
display(summary_df)

# Peek a couple frames
for i, df in enumerate(dfs[:2], 1):
    print(f"\n=== Preview #{i}: {df.attrs.get('source_file','UNKNOWN')} ===")
    display(df.head(10))


Found 125 files to load.
Loaded 125 dataframes.


,file,rows,cols,date_min,date_max
0,atp_matches_1968.csv,4377,49,1970-01-01 00:00:00.019671228+00:00,1970-01-01 00:00:00.019690101+00:00
1,atp_matches_1969.csv,3165,49,1970-01-01 00:00:00.019681230+00:00,1970-01-01 00:00:00.019691226+00:00
2,atp_matches_1970.csv,3287,49,1970-01-01 00:00:00.019700104+00:00,1970-01-01 00:00:00.019701226+00:00
3,atp_matches_1971.csv,3640,49,1970-01-01 00:00:00.019710110+00:00,1970-01-01 00:00:00.019711213+00:00
4,atp_matches_1972.csv,3617,49,1970-01-01 00:00:00.019711206+00:00,1970-01-01 00:00:00.019730116+00:00
...,...,...,...,...,...
120,atp_matches_futures_2022.csv,16306,49,1970-01-01 00:00:00.020220103+00:00,1970-01-01 00:00:00.020221226+00:00
121,atp_matches_futures_2023.csv,17626,49,1970-01-01 00:00:00.020230102+00:00,1970-01-01 00:00:00.020231225+00:00
122,atp_matches_futures_2023.csv,17626,49,1970-01-01 00:00:00.020230102+00:00,1970-01-01 00:00:00.020231225+00:00
123,atp_matches_futures_2024.csv,18423,49,1970-01-01 00:00:00.020240101+00:00,1970-01-01 00:00:00.020241216+00:00



=== Preview #1: atp_matches_1968.csv ===


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32.0,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32.0,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32.0,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32.0,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32.0,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1968-2029,Dublin,Grass,32.0,A,19680708,275,207073,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1968-2029,Dublin,Grass,32.0,A,19680708,276,109783,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1968-2029,Dublin,Grass,32.0,A,19680708,277,109745,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1968-2029,Dublin,Grass,32.0,A,19680708,278,201749,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1968-2029,Dublin,Grass,32.0,A,19680708,279,209525,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



=== Preview #2: atp_matches_1969.csv ===


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1969-2029,Dublin,Grass,32,A,19690707,270,100011,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1969-2029,Dublin,Grass,32,A,19690707,271,100058,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1969-2029,Dublin,Grass,32,A,19690707,272,100114,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1969-2029,Dublin,Grass,32,A,19690707,273,125672,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1969-2029,Dublin,Grass,32,A,19690707,274,100037,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1969-2029,Dublin,Grass,32,A,19690707,275,100067,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1969-2029,Dublin,Grass,32,A,19690707,276,207073,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1969-2029,Dublin,Grass,32,A,19690707,277,100120,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1969-2029,Dublin,Grass,32,A,19690707,278,110957,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1969-2029,Dublin,Grass,32,A,19690707,279,109783,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
if not dfs:
    raise RuntimeError("No dataframes loaded. Run the loading cell first.")

# Keep only real DataFrames (paranoia) and allow empty ones
dfs_clean = [d for d in dfs if isinstance(d, pd.DataFrame)]

all_matches = pd.concat(dfs_clean, ignore_index=True, sort=False)

# Normalize a date column if present
for candidate in ["tourney_date","date","match_date","start_date","utc_date"]:
    if candidate in all_matches.columns:
        all_matches[candidate] = pd.to_datetime(all_matches[candidate], errors="coerce", utc=True)
        break

# Drop exact duplicates (or provide a subset if you have a unique key)
all_matches = all_matches.drop_duplicates().reset_index(drop=True)



In [44]:
print(f"Merged shape: {c.shape}")
display(all_matches.sample(20))

Merged shape: (693551, 49)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
202480,1991-M-SA-MEX-04A-1991b,Mexcio 3 Masters 2,Hard,32.0,S,1970-01-01 00:00:00.019910930+00:00,22,101156,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,317.0,71.0,500.0,29.0
376591,2005-M-FU-FRA-19A-2005,France F19,Hard,32.0,S,1970-01-01 00:00:00.020051024+00:00,25,103424,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,755.0,19.0,372.0,81.0
494633,2013-M-FU-CHI-02A-2013,Chile F2,Clay,32.0,S,1970-01-01 00:00:00.020130422+00:00,12,105635,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,808.0,18.0,357.0,112.0
74505,1987-425,Barcelona,Clay,56.0,A,1970-01-01 00:00:00.019870921+00:00,2,100809,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,260.0,NaN,218.0,NaN
31222,1976-278,Barcelona,Clay,64.0,A,1970-01-01 00:00:00.019760404+00:00,39,100185,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73403,1987-375,Lyon,Carpet,32.0,A,1970-01-01 00:00:00.019870202+00:00,28,100893,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,340.0,NaN
73023,1987-328,Basel,Hard,32.0,A,1970-01-01 00:00:00.019871005+00:00,13,100434,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,78.0,NaN
652633,2022-M-ITF-SUI-04A-2022,M25 Muttenz,Clay,32.0,25,1970-01-01 00:00:00.020220815+00:00,402,104586,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,336.0,135.0,436.0,91.0
36645,1977-407,Rotterdam,Carpet,16.0,A,1970-01-01 00:00:00.019770321+00:00,12,100119,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,32.0,NaN
534060,2015-M-FU-ISR-01A-2015,Israel F1,Hard,32.0,S,1970-01-01 00:00:00.020150309+00:00,1,104993,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,308.0,153.0,NaN,NaN


In [49]:
all_matches.to_csv("data/all_matches.csv", index=False)

In [50]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693551 entries, 0 to 693550
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   tourney_id          693551 non-null  object             
 1   tourney_name        693551 non-null  object             
 2   surface             690561 non-null  object             
 3   draw_size           692716 non-null  float64            
 4   tourney_level       693551 non-null  object             
 5   tourney_date        693551 non-null  datetime64[ns, UTC]
 6   match_num           693551 non-null  int64              
 7   winner_id           693551 non-null  int64              
 8   winner_seed         303254 non-null  float64            
 9   winner_entry        123398 non-null  object             
 10  winner_name         693551 non-null  object             
 11  winner_hand         693548 non-null  object             
 12  winner_ht       

In [4]:
import pandas as pd
all_matches = pd.read_csv("data/all_matches.csv")

required_cols = [
    "tourney_id", "tourney_name", "surface", "draw_size", "tourney_level",
    "tourney_date", "winner_id", "winner_seed", "winner_name", "loser_name",
    "score", "w_ace", "l_ace", "w_1stIn", "w_1stWon", "w_2ndWon", "w_bpSaved",
    "w_bpFaced", "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon",
    "l_bpSaved", "l_bpFaced", "winner_rank", "loser_rank"
]

# Make sure we only include columns that actually exist in all_matches
required_cols = [c for c in required_cols if c in all_matches.columns]

# Filter DataFrame: keep only rows where all required columns are not null
filtered_matches = all_matches[all_matches[required_cols].notna().all(axis=1)].copy()

print(f"Original shape: {all_matches.shape}")
print(f"Filtered shape (no nulls in required columns): {filtered_matches.shape}")

# Preview
filtered_matches.to_csv("data/matches_with_stats.csv")
display(filtered_matches.head(20))


/tmp/ipykernel_205647/2915505224.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_matches = pd.read_csv("data/all_matches.csv")


Original shape: (693551, 49)
Filtered shape (no nulls in required columns): (43007, 49)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
86624,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,4,101889,8.0,NaN,...,45.0,30.0,11.0,10.0,5.0,8.0,50.0,NaN,84.0,NaN
86629,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,9,101529,6.0,NaN,...,42.0,27.0,13.0,10.0,7.0,9.0,42.0,NaN,67.0,NaN
86636,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,16,101793,2.0,NaN,...,51.0,36.0,13.0,12.0,11.0,15.0,25.0,NaN,86.0,NaN
86641,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,21,101529,6.0,NaN,...,42.0,26.0,7.0,9.0,2.0,6.0,42.0,NaN,66.0,NaN
86644,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,24,101793,2.0,NaN,...,36.0,21.0,14.0,10.0,6.0,10.0,25.0,NaN,54.0,NaN
86647,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,27,101529,6.0,NaN,...,57.0,39.0,12.0,13.0,4.0,7.0,42.0,NaN,60.0,NaN
86648,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,28,101793,2.0,NaN,...,34.0,28.0,12.0,10.0,1.0,3.0,25.0,NaN,111.0,NaN
86650,1991-339,Adelaide,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,30,101529,6.0,NaN,...,40.0,29.0,18.0,11.0,1.0,3.0,42.0,NaN,25.0,NaN
86655,1991-354,Wellington,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,4,101439,8.0,NaN,...,61.0,38.0,7.0,13.0,4.0,10.0,70.0,NaN,263.0,NaN
86656,1991-354,Wellington,Hard,32.0,A,1970-01-01 00:00:00.019901231+00:00,5,101735,3.0,NaN,...,40.0,17.0,13.0,10.0,7.0,13.0,32.0,NaN,103.0,NaN
